In [1]:
import numpy as np
import scipy.optimize as optimize

# Problem statement as Markov Pocess

Given the context of 1 day ( free places and demand) find the next optimal movements between hospitals.


Objective: minimize the number of unattended people by predicting a transition matrix of patient transports between hospitals

Prevent unnecessary transportantions between hospitals.

In [2]:
def optimize_func(transitions):
    n = int(np.sqrt(len(transitions)))
    transitions = transitions.reshape(n,n)
    new_availability = free_places - transitions.sum(axis = 0)
    nb_deaths = -np.sum(new_availability[new_availability<0])
    
     
    # penalize useless transportation
    occupied_in_hospital = np.array([transitions[i][i] for i in range(n)])
    remaining_in_hospital = free_places - occupied_in_hospital
    transported = transitions.sum(axis = 1) - occupied_in_hospital
    
    useless_transport = remaining_in_hospital * transported
    idx_tranport_not_needed = np.where(remaining_in_hospital > 0)[0]
    if len(idx_tranport_not_needed) > 0: # there are places left in the hospital
        # and still a transport has been done
        useless_transport = np.sum(useless_transport[idx_tranport_not_needed])
    else:
        useless_transport = 0
#     print(f"useless_transport {useless_transport}, remaining_in_hospital{remaining_in_hospital}, {transported}")
    result = nb_deaths + useless_transport
    return result


def demand_is_split_across_hospitals(transitions):
    n = int(np.sqrt(len(transitions)))
    transitions = transitions.reshape(n, n)
    diff = transitions.sum(axis=1) - demand
    result = np.sum(np.abs(diff))
    return result

constraints = (
    {
        'type': 'eq', 'fun': demand_is_split_across_hospitals
    } )

In [3]:
def run(demand, free_places):
    n = len(demand)
    transitions = np.zeros(n*n)
    bounds=[(0, np.sum(demand)) for _ in range(n*n)]

    result = optimize.minimize(optimize_func,
                               transitions,
                               method='SLSQP',
                               constraints=constraints,
                               bounds = bounds
                              )
#     print(result.fun)
    result = result.x
    return result.reshape(n,n).round().astype(int)

# Test

In [4]:
free_places = np.array([7,1,2])
demand = np.array([2,4,2])

run(demand, free_places)

array([[2, 0, 0],
       [3, 1, 0],
       [0, 0, 2]])

In [6]:
n = 4 # number of hospitals
free_places = np.random.randint(0, 5, size = n)
demand = np.random.randint(0, 5, size = n)
print ( f"free_places {free_places}, demand {demand} ")
run(demand, free_places)

free_places [1 0 1 0], demand [0 1 1 1] 


array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

In [7]:
n = 6 # number of hospitals
free_places = np.random.randint(0, 5, size = n)
demand = np.random.randint(0, 5, size = n)
print ( f"free_places {free_places}, demand {demand} ")
run(demand, free_places)

free_places [1 2 0 2 2 3], demand [4 2 3 4 1 0] 


array([[1, 0, 0, 0, 0, 1],
       [0, 2, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 2],
       [0, 0, 0, 2, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]])